## Introduction
In this document, the extreme value tests are conducted for 10 variables. Each time, the value is doubled or halved/set to zero, to see if the model breaks and if the model behaves as expected. The model runs for 25 years, with 10 iterations with different seeds. 
Per variable, the model will be run with three different input values: with low, normal, and high values. This is done by calling the function "run_scenario_full" with the different values. 

In [ ]:
from Model_for_sensitivity import RiverDeltaModel
import matplotlib.pyplot as plt
import networkx as nx
import warnings 
import copy
import matplotlib.lines as mlines
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

## SALINITY
Salinity low means there is 0 salinity at all, salinity high means that salinity is twice as high as normal

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, salinity_low, salinity_high, n_runs=10, max_steps=300):
    results = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, salinity_low=salinity_low, salinity_high=salinity_high)

        for _ in range(301):
            model.step()

        # Agent data
        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        crop_counts = step300_agents["Crop_type"].value_counts()

        # Model data
        # Model-level data
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        if max_steps in model_data.index:
            step_model_data = model_data.loc[[max_steps]]
        else:
            step_model_data = model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            # Agent-level info
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

    return pd.DataFrame(results)


In [ ]:
# Call function to run the analysis
df_low = run_scenario_full("Low salinity", salinity_low=True, salinity_high=False)
df_medium = run_scenario_full("Medium salinity", salinity_low=False, salinity_high=False)
df_high = run_scenario_full("High salinity", salinity_low=False, salinity_high=True)

# Put all output variables in a dataframe
all_full = pd.concat([df_low, df_medium, df_high], ignore_index=True)


In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_full
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()
all_full_grouped

In [ ]:
# Select correct variables
crop_long = all_full_grouped.melt(
    id_vars=["Scenario"],
    value_vars=["Rice", "Annual crops", "Perennial crops", "Aquaculture"],
    var_name="Crop Type",
    value_name="Count"
)

# Implement a desired order to make the graph easy to read and understand
desired_order = ["Low salinity", "Medium salinity", "High salinity"]
crop_long["Scenario"] = pd.Categorical(crop_long["Scenario"], categories=desired_order, ordered=True)

# Make sure that there are relative values
def normalize(group):
    medium_mean = group.loc[group["Scenario"] == "Medium salinity", "Count"].mean()
    group["Relative Count"] = group["Count"] / medium_mean if medium_mean != 0 else 0
    return group

crop_long = crop_long.groupby("Crop Type").apply(normalize)

# Boxplot with the relative values
plt.figure(figsize=(10, 6))
sns.boxplot(data=crop_long, x="Crop Type", y="Relative Count", hue="Scenario", hue_order=desired_order)
plt.axhline(1.0, linestyle="--", color="gray", label="Normal salinity = 1.0")
plt.title("Relative number of agents per crop type, compared to normal salinity levels")
plt.ylabel("Relative number, compared to normal salinity")
plt.xlabel("Crop Type")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


In [ ]:
# Set a predefined order to make the graph easy to understand
desired_order = ["Low salinity", "Medium salinity", "High salinity"]
# Calculate the mean to make it possible to compare
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Medium salinity", "Migrated_households"
].mean()

# Create a dataframe with the normalized values
normalized_df = all_full_grouped.copy()
normalized_df["Normalized_Migrated"] = normalized_df["Migrated_households"] / medium_mean

# Create the boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=normalized_df,
    x="Scenario",
    y="Normalized_Migrated",
    hue="Scenario",
    order = desired_order,
    palette="Set2"
)
plt.axhline(1, linestyle="--", color="gray", label="Gem. Medium salinity")

plt.title("Normalized distribution of migrated households during different salinity levels")
plt.xlabel("Scenario")
plt.ylabel("Normalized number of migrated households, compared to normal salinity")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


## SALINITY SHOCKS
In the base case, there are shocks in 2016, 2020, 2030, 2034, 2038. But what happens if there are no shocks at all, or there are twice as many shocks?

In [ ]:
# Create function to run the three options (no, normal, twice as many)
def run_scenario_full(scenario_name, salinity_shock_times, n_runs=10, max_steps=300):
    results = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, salinity_shock_step = salinity_shock_times)

        for _ in range(301):
            model.step()

        # Agent data
        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        crop_counts = step300_agents["Crop_type"].value_counts()

        # Model data
        # Model-level data
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        if max_steps in model_data.index:
            step_model_data = model_data.loc[[max_steps]]
        else:
            step_model_data = model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            # Agent-level info
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

    return pd.DataFrame(results)


In [ ]:
# Call function to run the analysis
df_low = run_scenario_full("No shocks", salinity_shock_times = [])
df_medium = run_scenario_full("Normal shocks", salinity_shock_times = [25,49,145,193,241,289])
df_high = run_scenario_full("Twice as many shocks", salinity_shock_times = [25,37,49,133,145,181,193,205,241,253,289])

# Put all output variables in a dataframe
all_full = pd.concat([df_low, df_medium, df_high], ignore_index=True)


In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_full
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()
all_full_grouped

In [ ]:
# Select correct variables
crop_long = all_full_grouped.melt(
    id_vars=["Scenario"],
    value_vars=["Rice", "Annual crops", "Perennial crops", "Aquaculture"],
    var_name="Crop Type",
    value_name="Count"
)

# Implement a desired order to make the graph easy to read and understand
desired_order = ["No shocks", "Normal shocks", "Twice as many shocks"]
crop_long["Scenario"] = pd.Categorical(crop_long["Scenario"], categories=desired_order, ordered=True)

# Make sure that there are relative values
def normalize(group):
    medium_mean = group.loc[group["Scenario"] == "Normal shocks", "Count"].mean()
    group["Relative Count"] = group["Count"] / medium_mean if medium_mean != 0 else 0
    return group

crop_long = crop_long.groupby("Crop Type").apply(normalize)

# Boxplot with the relative values
plt.figure(figsize=(10, 6))
sns.boxplot(data=crop_long, x="Crop Type", y="Relative Count", hue="Scenario", hue_order=desired_order)
plt.axhline(1.0, linestyle="--", color="gray", label="Normal number of shocks")
plt.title("Relative number of agents per crop type, based on normal number of shocks")
plt.ylabel("Relative number of agents, compared to normal number of shocks")
plt.xlabel("Crop Type")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


In [ ]:
# Look at the mean number of migrated households when there is a normal amount of shocks
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal shocks", "Migrated_households"
].mean()

# Divide the number of migrated households by the mean number of migrated households
all_full_grouped["Migrated_households_norm"] = all_full_grouped["Migrated_households"] / medium_mean

# Boxplot with the relative values
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=all_full_grouped,
    x="Scenario",
    y="Migrated_households_norm",
    hue="Scenario",
    palette="Set2"
)

plt.axhline(1, linestyle="--", color="gray", label="Normal shocks")

plt.title("Nonrmalized distribution of migrated households during salinity shocks")
plt.xlabel("Scenario")
plt.ylabel("Migrated households compared to normal level of salinity shocks")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()



## PRODUCTION COSTS 
Set the production costs to zero, or make them twice as high

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, production_costs, n_runs=10, max_steps=300):
    results = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, production_costs_scenario=production_costs)

        for _ in range(301):
            model.step()

        # Agent data
        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        crop_counts = step300_agents["Crop_type"].value_counts()

        # Model data
        # Model-level data
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        if max_steps in model_data.index:
            step_model_data = model_data.loc[[max_steps]]
        else:
            step_model_data = model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            # Agent-level info
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

    return pd.DataFrame(results)


In [ ]:
# Call function to run the analysis
df_low = run_scenario_full("Low production costs", production_costs = "Low")
df_medium = run_scenario_full("Normal production costs", production_costs = "Medium")
df_high = run_scenario_full("High production costs", production_costs = "High")

# Put all output variables in a dataframe
all_full = pd.concat([df_low, df_medium, df_high], ignore_index=True)

In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_full
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()
all_full_grouped

In [ ]:
# Select the correct values
crop_long = all_full_grouped.melt(
    id_vars=["Scenario"],
    value_vars=["Rice", "Annual crops", "Perennial crops", "Aquaculture"],
    var_name="Crop Type",
    value_name="Count"
)

# Implement a desired order to make the graph easy to read and understand
desired_order = ["Low production costs", "Normal production costs", "High production costs"]
crop_long["Scenario"] = pd.Categorical(crop_long["Scenario"], categories=desired_order, ordered=True)

# Make sure that there are relative values
def normalize(group):
    medium_mean = group.loc[group["Scenario"] == "Normal production costs", "Count"].mean()
    group["Relative Count"] = group["Count"] / medium_mean if medium_mean != 0 else 0
    return group

crop_long = crop_long.groupby("Crop Type").apply(normalize)

# Boxplot with the relative values
plt.figure(figsize=(10, 6))
sns.boxplot(data=crop_long, x="Crop Type", y="Relative Count", hue="Scenario", hue_order=desired_order)
plt.axhline(1.0, linestyle="--", color="gray", label="Normal production costs")
plt.title("Number of agents per crop type, when production costs are zero, normal or twice as high")
plt.ylabel("Relative number of agents, compared to normal production costs")
plt.xlabel("Crop Type")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()

In [ ]:
# Look at the mean number of migrated households when there are normal production costs
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal production costs", "Migrated_households"
].mean()

# Set a desired order to make the graph easy to understand
desired_order = ["Low production cost", "Normal production cost", "High production cost"]

# Divide the number of migrated households by the mean number of migrated households
all_full_grouped["Migrated_households_norm"] = all_full_grouped["Migrated_households"] / medium_mean

# Create boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=all_full_grouped,
    x="Scenario",
    y="Migrated_households_norm",
    hue="Scenario",
    order = desired_order,
    palette="Set2"
)

plt.axhline(1, linestyle="--", color="gray", label="Average production costs")

plt.title("Normalized distribution of migrated households, using different production costs")
plt.xlabel("Scenario")
plt.ylabel("Normalized number of migrated households, compared to normal production costs")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


## WAGE WORKERS
It is checked what will happen with the total number of wage workers when they earn twice as much, or only half of their current salary. 
Second, it is checked what will happen when the farmers need only half the man days/ha/season, or twice as much. 

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, ww_hourly_wage, n_runs=10, max_steps=300):
    results = []

    all_agent_data = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, wage_of_ww=ww_hourly_wage)

        for _ in range(301):
            model.step()

        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        step300_agents["Scenario"] = scenario_name
        step300_agents["Seed"] = seed
        all_agent_data.append(step300_agents)

        crop_counts = step300_agents["Crop_type"].value_counts()
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        step_model_data = model_data.loc[[max_steps]] if max_steps in model_data.index else model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

 
    summary_df = pd.DataFrame(results)
    agents_df = pd.concat(all_agent_data, ignore_index=True)
    return summary_df, agents_df



In [ ]:
# Call function to run the analysis
summary_low, agents_low = run_scenario_full("Low wage", 0.5)
summary_medium, agents_medium = run_scenario_full("Medium wage", 1)
summary_high, agents_high = run_scenario_full("High wage", 2)

# Put all output variables in a dataframe 
# There are two dataframes this time. One for the agent data (based on the agent metrics in the model) and one for the other results (e.g. number of farmers having Rice)
all_summary = pd.concat([summary_low, summary_medium, summary_high], ignore_index=True)
all_agents = pd.concat([agents_low, agents_medium, agents_high], ignore_index=True)

In [ ]:
# Make sure there is numerical data, to take the mean
all_agents["too low income"] = pd.to_numeric(all_agents["too low income"], errors="coerce")

# Implement a desired order to make the graph easy to read and understand
desired_order = ["Low wage", "Medium wage", "High wage"]

# Take the sum of the total households with too low income
# Group by scenario
too_low_counts = (
    all_agents.groupby(["Scenario", "Seed"])["too low income"]
    .sum()
    .reset_index(name="TooLowCount")
)

mean_per_scenario = too_low_counts.groupby("Scenario")["TooLowCount"].mean()

# Add a baseline to compare
baseline = mean_per_scenario["Medium wage"]
too_low_counts["RelativeTooLow"] = too_low_counts.apply(
    lambda row: row["TooLowCount"] / baseline, axis=1
)

# Create boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(data=too_low_counts, x="Scenario", y="RelativeTooLow", order = desired_order)
plt.axhline(1, linestyle="--", color="gray", label="Baseline (Salary is normal)")
plt.title("Relative number of household members with too low income")
plt.ylabel("Relative number of household members compared to normal salary")
plt.xlabel("Scenario")
plt.legend()
plt.tight_layout()
plt.show()




In [ ]:
# Implement a desired order to make the graph easy to read and understand
desired_order = ["Low wage", "Medium wage", "High wage"]
# Set to "number of wage worker column" to numeric to make it possible to take the mean
all_agents["Number_of_wage_workers"] = pd.to_numeric(all_agents["Number_of_wage_workers"], errors="coerce")

# Take the sum of the total households with too low income
# Group by scenario
too_low_counts = (
    all_agents.groupby(["Scenario", "Seed"])["Number_of_wage_workers"]
    .mean()
    .reset_index(name="TooLowCount")
)

mean_per_scenario = too_low_counts.groupby("Scenario")["TooLowCount"].mean()

# Add a baseline to compare
baseline = mean_per_scenario["Medium wage"]
too_low_counts["RelativeTooLow"] = too_low_counts.apply(
    lambda row: row["TooLowCount"] / baseline, axis=1
)

# Create boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(data=too_low_counts, x="Scenario", y="RelativeTooLow", order=desired_order)
plt.axhline(1, linestyle="--", color="gray", label="Baseline (Salary is normal)")
plt.title("Relative number of wage workers, when salary is changed")
plt.ylabel("Relative number of households")
plt.xlabel("Scenario")
plt.legend()
plt.tight_layout()
plt.show()


Now it is checked what will happen if only half of the man days / ha is needed, or twice as much. When more workers are needed, it is expected that there will be less farmers (they cannot pay for the wage workers), and therefore also less wage workers?  

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, required_ww, n_runs=10, max_steps=300):
    results = []

    all_agent_data = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, wage_workers_required=required_ww)

        for _ in range(301):
            model.step()

        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        step300_agents["Scenario"] = scenario_name
        step300_agents["Seed"] = seed
        all_agent_data.append(step300_agents)

        crop_counts = step300_agents["Crop_type"].value_counts()
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        step_model_data = model_data.loc[[max_steps]] if max_steps in model_data.index else model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

    summary_df = pd.DataFrame(results)
    agents_df = pd.concat(all_agent_data, ignore_index=True)
    return summary_df, agents_df



In [ ]:
# Call function to run the analysis
summary_low, agents_low = run_scenario_full("Low number needed", 0.5)
summary_medium, agents_medium = run_scenario_full("Medium number needed", 1)
summary_high, agents_high = run_scenario_full("High number needed", 2)

# Put all output variables in a dataframe 
# There are two dataframes this time. One for the agent data (based on the agent metrics in the model) and one for the other results (e.g. number of farmers having Rice)
all_summary = pd.concat([summary_low, summary_medium, summary_high], ignore_index=True)
all_agents = pd.concat([agents_low, agents_medium, agents_high], ignore_index=True)



In [ ]:
# Set data to numeric to make it possible to look at the mean
all_agents["Number_of_wage_workers"] = pd.to_numeric(all_agents["Number_of_wage_workers"], errors="coerce")

# Take the sum of the total households with too low income
# Group by scenario
too_low_counts = (
    all_agents.groupby(["Scenario", "Seed"])["Number_of_wage_workers"]
    .mean()
    .reset_index(name="TooLowCount")
)


mean_per_scenario = too_low_counts.groupby("Scenario")["TooLowCount"].mean()

# Add a baseline to compare
baseline = mean_per_scenario["Medium number needed"]
too_low_counts["RelativeTooLow"] = too_low_counts.apply(
    lambda row: row["TooLowCount"] / baseline, axis=1
)

# Create boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(data=too_low_counts, x="Scenario", y="RelativeTooLow", order=['Low number needed', "Medium number needed", "High number needed"])
plt.axhline(1, linestyle="--", color="gray", label="Baseline (Medium = 1)")
plt.title("Number of wage workers when different numbers of wage workers are required")
plt.ylabel("Relative number of wage workers compared to normal scenario")
plt.xlabel("Scenario")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_summary
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()
all_full_grouped

In [ ]:
# Make sure that the relevant values are selected
crop_long = all_full_grouped.melt(
    id_vars=["Scenario"],
    value_vars=["Rice", "Annual crops", "Perennial crops", "Aquaculture"],
    var_name="Crop Type",
    value_name="Count"
)

# Implement a desired order to make the graph easy to read and understand
desired_order = ["Low number needed", "Medium number needed", "High number needed"]
crop_long["Scenario"] = pd.Categorical(crop_long["Scenario"], categories=desired_order, ordered=True)

# Make sure that there are relative values
def normalize(group):
    medium_mean = group.loc[group["Scenario"] == "Medium number needed", "Count"].mean()
    group["Relative Count"] = group["Count"] / medium_mean if medium_mean != 0 else 0
    return group

crop_long = crop_long.groupby("Crop Type").apply(normalize)

# Boxplot with the relative values
plt.figure(figsize=(10, 6))
sns.boxplot(data=crop_long, x="Crop Type", y="Relative Count", hue="Scenario", hue_order=desired_order)
plt.axhline(1.0, linestyle="--", color="gray", label="Normal wage workers required")
plt.title("Relative number of farmers when required number of wage workers changes")
plt.ylabel("Relative number of farmers, comopared to normal scenario")
plt.xlabel("Crop Type")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


## ACCESS TO THE INFORMATION MEETING

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, information_meeting, n_runs=10, max_steps=300):
    results = []

    all_agent_data = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, chance_info_meeting=information_meeting)

        for _ in range(301):
            model.step()

        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        step300_agents["Scenario"] = scenario_name
        step300_agents["Seed"] = seed
        all_agent_data.append(step300_agents)

        crop_counts = step300_agents["Crop_type"].value_counts()
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        step_model_data = model_data.loc[[max_steps]] if max_steps in model_data.index else model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

    summary_df = pd.DataFrame(results)
    agents_df = pd.concat(all_agent_data, ignore_index=True)
    return summary_df, agents_df



In [ ]:
# Call function to run the analysis
summary_low, agents_low = run_scenario_full("Nobody attends", 0)
summary_medium, agents_medium = run_scenario_full("Normal attendance ", 0.1)
summary_high, agents_high = run_scenario_full("High attendance", 1)

# Put all output variables in a dataframe 
# There are two dataframes this time. One for the agent data (based on the agent metrics in the model) and one for the other results (e.g. number of farmers having Rice)
all_summary = pd.concat([summary_low, summary_medium, summary_high], ignore_index=True)
all_agents = pd.concat([agents_low, agents_medium, agents_high], ignore_index=True)



In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_summary
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()


In [ ]:
# Select relevant values
crop_long = all_full_grouped.melt(
    id_vars=["Scenario"],
    value_vars=["Rice", "Annual crops", "Perennial crops", "Aquaculture"],
    var_name="Crop Type",
    value_name="Count"
)

# Implement a desired order to make the graph easy to read and understand
desired_order = ["Nobody attends", "Normal attendance ", "High attendance"]
crop_long["Scenario"] = pd.Categorical(crop_long["Scenario"], categories=desired_order, ordered=True)

# Make sure that there are relative values
def normalize(group):
    medium_mean = group.loc[group["Scenario"] == "Normal attendance ", "Count"].mean()
    group["Relative Count"] = group["Count"] / medium_mean if medium_mean != 0 else 0
    return group

crop_long = crop_long.groupby("Crop Type").apply(normalize)

# Boxplot with the relative values
plt.figure(figsize=(10, 6))
sns.boxplot(data=crop_long, x="Crop Type", y="Relative Count", hue="Scenario", hue_order=desired_order)
plt.axhline(1.0, linestyle="--", color="gray", label="Normal attendence = 1.0")
plt.title("Relative number of households per crop type, compared to normal attendence to information meetings")
plt.ylabel("Relative number of households")
plt.xlabel("Crop Type")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


In [ ]:
all_agents.head()

In [ ]:
# Take the mean of the average livelihood with normal attendance

medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal attendance ", "Average_Livelihood"
].mean()

# Divide the number average livelihood per run by the average livelihood in normal attendance
all_full_grouped["Migrated_households_norm"] = all_full_grouped["Average_Livelihood"] / medium_mean

# Plot figure
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=all_full_grouped,
    x="Scenario",
    y="Migrated_households_norm",
    hue="Scenario",
    order = ['Nobody attends', "Normal attendance ", "High attendance"],
    palette="Set2"
)

# Add baseline
plt.axhline(1, linestyle="--", color="gray", label="Average information meeting attendence")

plt.title("Normalized livelihood of land households")
plt.xlabel("Scenario")
plt.ylabel("Relative livelihood compared to normal information meeting attendence")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()



### CONTACTS IN THE CITY
what happens if everybody has contacts in the city, or nobody?

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, contacts_in_city, n_runs=10, max_steps=300):
    results = []

    all_agent_data = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, scenario_contacts=contacts_in_city)

        for _ in range(301):
            model.step()

        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        step300_agents["Scenario"] = scenario_name
        step300_agents["Seed"] = seed
        all_agent_data.append(step300_agents)

        crop_counts = step300_agents["Crop_type"].value_counts()
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        step_model_data = model_data.loc[[max_steps]] if max_steps in model_data.index else model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

    summary_df = pd.DataFrame(results)
    agents_df = pd.concat(all_agent_data, ignore_index=True)
    return summary_df, agents_df



In [ ]:
# Call function to run the analysis
summary_low, agents_low = run_scenario_full("Zero contacts", "Low")
summary_medium, agents_medium = run_scenario_full("Normal contacts ", "Normal")
summary_high, agents_high = run_scenario_full("Everyone has contacts", "High")

# Put all output variables in a dataframe 
# There are two dataframes this time. One for the agent data (based on the agent metrics in the model) and one for the other results (e.g. number of farmers having Rice)
all_summary = pd.concat([summary_low, summary_medium, summary_high], ignore_index=True)
all_agents = pd.concat([agents_low, agents_medium, agents_high], ignore_index=True)



In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_summary
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()

In [ ]:
# Calculate the mean number of migrated individuals when there is a normal number of contacts
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal contacts ", "Migrated_individuals"
].mean()

# Divide all data by this mean
normalized_df = all_full_grouped.copy()
normalized_df["Normalized_Migrated"] = normalized_df["Migrated_individuals"] / medium_mean

# Create a boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=normalized_df,
    x="Scenario",
    y="Normalized_Migrated",
    hue="Scenario",
    palette="Set2"
)


plt.axhline(1, linestyle="--", color="gray", label="Average when there are normal contacts")

plt.title("Number of migrated houehold members, when the probability for contacts in the city changes")
plt.xlabel("Scenario")
plt.ylabel("Normalized number of individual household members")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


## DEBT
Nobody can get a loan, or the maximal debts (value of assets) is twice is high
dit geeft niet zoveel effect, kijk naar de grafiek van de debt ratio over time, en zie dat deze erg laag is

In [ ]:
# Create function to run the three options (no, normal, high)
def run_scenario_full(scenario_name, debt_multiplier, n_runs=10, max_steps=300):
    results = []

    all_agent_data = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, debt_scenario=debt_multiplier)

        for _ in range(301):
            model.step()

        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        step300_agents["Scenario"] = scenario_name
        step300_agents["Seed"] = seed
        all_agent_data.append(step300_agents)

        crop_counts = step300_agents["Crop_type"].value_counts()
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        step_model_data = model_data.loc[[max_steps]] if max_steps in model_data.index else model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)


    summary_df = pd.DataFrame(results)
    agents_df = pd.concat(all_agent_data, ignore_index=True)
    return summary_df, agents_df



In [ ]:
# Call function to run the analysis
summary_low, agents_low = run_scenario_full("No debt", 0.001)
summary_medium, agents_medium = run_scenario_full("Normal debt", 1)
summary_high, agents_high = run_scenario_full("High debt", 2)

# Put all output variables in dataframes
all_summary = pd.concat([summary_low, summary_medium, summary_high], ignore_index=True)
all_agents = pd.concat([agents_low, agents_medium, agents_high], ignore_index=True)



In [ ]:
# Set savings to numeric to make it possible to take the mean
all_agents["Savings"] = pd.to_numeric(all_agents["Savings"], errors="coerce")

# Take the mean savings per scenario, and groupby scenario
too_low_counts = (
    all_agents.groupby(["Scenario", "Seed"])["Savings"]
    .mean()
    .reset_index(name="TooLowCount")
)


mean_per_scenario = too_low_counts.groupby("Scenario")["TooLowCount"].mean()

# add a baseline based on the normal debt possibility, and create the boxplot
baseline = mean_per_scenario["Normal debt"]
too_low_counts["RelativeTooLow"] = too_low_counts.apply(
    lambda row: row["TooLowCount"] / baseline, axis=1
)

plt.figure(figsize=(8, 6))
sns.boxplot(data=too_low_counts, x="Scenario", y="RelativeTooLow")
plt.axhline(1, linestyle="--", color="gray", label="Baseline (Medium = 1)")
plt.title("Relative savings (Medium = 1)")
plt.ylabel("Relatief t.o.v. Medium")
plt.xlabel("Scenario")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_summary
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()


In [ ]:
# Calculate the average livelihood when there is normal debt
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal debt", "Average_Livelihood"
].mean()

# Divide all average livelihoods by the average livelihood calculated above
all_full_grouped["Migrated_households_norm"] = all_full_grouped["Average_Livelihood"] / medium_mean

# Create boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=all_full_grouped,
    x="Scenario",
    y="Migrated_households_norm",
    hue="Scenario",
    palette="Set2"
)

plt.axhline(1, linestyle="--", color="gray", label="Gem. normal debt")

plt.title("Genormaliseerde Verdeling van average livelihood per Scenario")
plt.xlabel("Scenario")
plt.ylabel("Livelihood, based on normal debt ")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()

In [ ]:
# Select relevant variables
crop_long = all_full_grouped.melt(
    id_vars=["Scenario"],
    value_vars=["Rice", "Annual crops", "Perennial crops", "Aquaculture"],
    var_name="Crop Type",
    value_name="Count"
)

# create a desired order to make the graph easy to understand
desired_order = ["No debt", "Normal debt", "High debt"]
crop_long["Scenario"] = pd.Categorical(crop_long["Scenario"], categories=desired_order, ordered=True)

# Make sure there are relative variables 
def normalize(group):
    medium_mean = group.loc[group["Scenario"] == "Normal debt", "Count"].mean()
    group["Relative Count"] = group["Count"] / medium_mean if medium_mean != 0 else 0
    return group

crop_long = crop_long.groupby("Crop Type").apply(normalize)

# create boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=crop_long, x="Crop Type", y="Relative Count", hue="Scenario", hue_order=desired_order)
plt.axhline(1.0, linestyle="--", color="gray", label="Normal debt = 1.0")
plt.title("Relative number of farmers per crop type, compared to normal possibility for debt")
plt.ylabel("Relative number of households, compared to normal debt")
plt.xlabel("Crop Type")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


In [ ]:

# Calculate the average number of households when there is normal debt
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal debt", "Migrated_households"
].mean()

# Divide all average livelihoods by the average livelihood calculated above
normalized_df = all_full_grouped.copy()
normalized_df["Normalized_Migrated"] = normalized_df["Migrated_households"] / medium_mean

# Create boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=normalized_df,
    x="Scenario",
    y="Normalized_Migrated",
    hue="Scenario",
    order = ['No debt', "Normal debt", "High debt"],
    palette="Set2"
)

# Add baseline
plt.axhline(1, linestyle="--", color="gray", label="Average migrations during medium debt")

plt.title("Normalized number of migrated households, with different debt levels ")
plt.xlabel("Scenario")
plt.ylabel("Normalized number of migrated households")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()


## PROBABILITY FOR MIGRATION

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, migration_chance, n_runs=10, max_steps=300):
    results = []

    all_agent_data = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, scenario_migration=migration_chance)

        for _ in range(301):
            model.step()

        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        step300_agents["Scenario"] = scenario_name
        step300_agents["Seed"] = seed
        all_agent_data.append(step300_agents)

        crop_counts = step300_agents["Crop_type"].value_counts()
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        step_model_data = model_data.loc[[max_steps]] if max_steps in model_data.index else model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)


    summary_df = pd.DataFrame(results)
    agents_df = pd.concat(all_agent_data, ignore_index=True)
    return summary_df, agents_df



In [ ]:
# Call function to run the analysis
summary_low, agents_low = run_scenario_full("Lower chance", 0.5)
summary_medium, agents_medium = run_scenario_full("Normal chance", 1)
summary_high, agents_high = run_scenario_full("Higher chance", 2)

# Put all output variables in a dataframe 
# There are two dataframes this time. One for the agent data (based on the agent metrics in the model) and one for the other results (e.g. number of farmers having Rice)
all_summary = pd.concat([summary_low, summary_medium, summary_high], ignore_index=True)
all_agents = pd.concat([agents_low, agents_medium, agents_high], ignore_index=True)

In [ ]:
all_full_grouped = all_summary
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()


In [ ]:
# Look at the mean number of migrated households when there is a normal chance of migrating
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal chance", "Migrated_households"
].mean()

# Divide all migrated households per scenario by the average number of migrated households in normal scenarios
normalized_df = all_full_grouped.copy()
normalized_df["Normalized_Migrated"] = normalized_df["Migrated_households"] / medium_mean

plt.figure(figsize=(8, 6))
sns.boxplot(
    data=normalized_df,
    x="Scenario",
    y="Normalized_Migrated",
    hue="Scenario",
    order= ['Lower chance', "Normal chance", "Higher chance"],
    palette="Set2"
)
# Add baseline
plt.axhline(1, linestyle="--", color="gray", label="Normal chance")

plt.title("Normalized number of migrated households, when changing migration probabilities")
plt.xlabel("Scenario")
plt.ylabel("Normalized number of migrated households")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()

In [ ]:
# Look at the mean number of migrated individuals when there is a normal chance of migrating
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal chance", "Migrated_individuals"
].mean()

# Divide all migrated individuals per scenario by the average number of migrated individuals in normal scenarios
normalized_df = all_full_grouped.copy()
normalized_df["Normalized_Migrated"] = normalized_df["Migrated_individuals"] / medium_mean

# Create boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=normalized_df,
    x="Scenario",
    y="Normalized_Migrated",
    hue="Scenario",
    order= ['Lower chance', "Normal chance", "Higher chance"],
    palette="Set2"
)

# Add baseline
plt.axhline(1, linestyle="--", color="gray", label="Normal chance")

plt.title("Normalized number of migrated household members, when changing migration probabilities")
plt.xlabel("Scenario")
plt.ylabel("Normalized number of migrated household members")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()

## FACILITIES IN NEIGHBORHOOD

In [ ]:
# Create function to run the three options (low, medium, high)
def run_scenario_full(scenario_name, number_of_facilities, n_runs=10, max_steps=300):
    results = []

    all_agent_data = []

    for seed in range(n_runs):
        model = RiverDeltaModel(seed=seed, scenario_facilities=number_of_facilities)

        for _ in range(301):
            model.step()

        agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
        step300_agents = agent_data[agent_data["Step"] == 300]
        step300_agents["Scenario"] = scenario_name
        step300_agents["Seed"] = seed
        all_agent_data.append(step300_agents)

        crop_counts = step300_agents["Crop_type"].value_counts()
        model_data = model.datacollector.get_model_vars_dataframe()
        model_data["Step"] = model_data.index
        step_model_data = model_data.loc[[max_steps]] if max_steps in model_data.index else model_data.tail(1)

        if not step_model_data.empty:
            data_row = step_model_data.iloc[0].to_dict()
        else:
            data_row = {}

        data_row.update({
            "Scenario": scenario_name,
            "Seed": seed,
        })

        results.append({
            "Scenario": scenario_name,
            "Seed": seed,
            "Rice": crop_counts.get("Rice", 0),
            "Annual crops": crop_counts.get("Annual crops", 0),
            "Perennial crops": crop_counts.get("Perennial crops", 0),
            "Aquaculture": crop_counts.get("Aquaculture", 0)
        })

        results.append(data_row)

   
    summary_df = pd.DataFrame(results)
    agents_df = pd.concat(all_agent_data, ignore_index=True)
    return summary_df, agents_df



In [ ]:
# Call function to run the analysis
summary_low, agents_low = run_scenario_full("Lower facilities", "Low")
summary_medium, agents_medium = run_scenario_full("Normal facilities", "Normal")
summary_high, agents_high = run_scenario_full("High facilities", "High")

# Put all output variables in a dataframe 
# There are two dataframes this time. One for the agent data (based on the agent metrics in the model) and one for the other results (e.g. number of farmers having Rice)
all_summary = pd.concat([summary_low, summary_medium, summary_high], ignore_index=True)
all_agents = pd.concat([agents_low, agents_medium, agents_high], ignore_index=True)

In [ ]:
# Fill in the nan values with 0, group by scenario and seed, and reset index
all_full_grouped = all_summary
all_full_grouped = all_full_grouped.fillna(0)
all_full_grouped = all_full_grouped.groupby(['Scenario', "Seed"]).sum()
all_full_grouped = all_full_grouped.reset_index()

In [ ]:
# Look at the mean number of migrated households when there is a normal amount of facilities
medium_mean = all_full_grouped.loc[
    all_full_grouped["Scenario"] == "Normal facilities", "Migrated_households"
].mean()

# Divide the migrated households by the average number of migrated households
normalized_df = all_full_grouped.copy()
normalized_df["Normalized_Migrated"] = normalized_df["Migrated_households"] / medium_mean

# Create a boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(
    data=normalized_df,
    x="Scenario",
    y="Normalized_Migrated",
    hue="Scenario",
    order = ['Lower facilities', 'Normal facilities', "High facilities"],
    palette="Set2"
)

# Add a baseline
plt.axhline(1, linestyle="--", color="gray", label="Gem. Normal facilities")

plt.title("Normalized number of migrated households, when changing number of facilities")
plt.xlabel("Scenario")
plt.ylabel("Normalized number of migrated households")
plt.legend(title="Scenario")
plt.tight_layout()
plt.show()